In [2]:
import torch
import transformers
from transformers import GenerationConfig, LlamaForCausalLM, LlamaTokenizer
import warnings
warnings.filterwarnings('ignore')

In [3]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
DEVICE

'cuda'

In [4]:
tokenizer = LlamaTokenizer.from_pretrained("decapoda-research/llama-7b-hf")

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'LLaMATokenizer'. 
The class this function is called from is 'LlamaTokenizer'.
normalizer.cc(51) LOG(INFO) precompiled_charsmap is empty. use identity normalization.


In [5]:
model = LlamaForCausalLM.from_pretrained(
    "decapoda-research/llama-7b-hf",
    load_in_8bit=True,
    device_map="auto",
)

Overriding torch_dtype=None with `torch_dtype=torch.float16` due to requirements of `bitsandbytes` to enable model loading in mixed int8. Either pass torch_dtype=torch.float16 or don't pass this argument at all to remove this warning.



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /home/chaewon/anaconda3/envs/Alpaca/lib/python3.7/site-packages/bitsandbytes/libbitsandbytes_cuda117.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 8.9
CUDA SETUP: Detected CUDA version 117
CUDA SETUP: Loading binary /home/chaewon/anaconda3/envs/Alpaca/lib/python3.7/site-packages/bitsandbytes/libbitsandbytes_cuda117.so...


Loading checkpoint shards:   0%|          | 0/33 [00:00<?, ?it/s]

In [6]:
from peft import PeftModel
model = PeftModel.from_pretrained(model, "tloen/alpaca-lora-7b", torch_dtype=torch.float16)

In [7]:
model.config.pad_token_id = tokenizer.pad_token_id = 0  # unk
model.config.bos_token_id = 1
model.config.eos_token_id = 2
 
model = model.eval()
# model = torch.compile(model)

In [8]:
def df_generate_prompt(x) -> str:
    if x["input"]:
        return f"""Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

              ### Instruction:
              {x["instruction"]}

              ### Input:
              {x["input"]}

              ### Response:"""
    else:
        return f"""Below is an instruction that describes a task. Write a response that appropriately completes the request.

                  ### Instruction:
                  {x["instruction"]}

                  ### Response:"""

In [9]:
import pandas as pd
import json
import os
# pd.options.display.max_colwidth = 2000

data = []
data_dir = os.path.join(os.getcwd(), "alpaca-lora/code_seed_dataset.jsonl")
with open(data_dir) as f:
  data.append(json.load(f))
data = data[0]

df = pd.DataFrame(data)
train_df = df.drop(columns=["output"])

train_df["prompt_input"] = train_df.apply(df_generate_prompt, axis=1)
train_df

,instruction,input,prompt_input
0,Does it have any filter rules ?,@pytest mark django dbdef test vfolder with no...,"Below is an instruction that describes a task,..."
1,Ca a range generate ranges with a length of mo...,def safe range *args rng xrange *args if len r...,"Below is an instruction that describes a task,..."
2,What ca a range nt generate ?,def safe range *args rng xrange *args if len r...,"Below is an instruction that describes a task,..."
3,What ca nt generate ranges with a length of mo...,def safe range *args rng xrange *args if len r...,"Below is an instruction that describes a task,..."
4,Do method decorator indicate a method is not a...,def not allowed func def inner self *args **kw...,"Below is an instruction that describes a task,..."
...,...,...,...
6995,What does the code return ?,def get index videos course return list {attr ...,"Below is an instruction that describes a task,..."
6996,What contains the substring specified by the a...,def assert has text output text assert output ...,"Below is an instruction that describes a task,..."
6997,What does specified output contain ?,def assert has text output text assert output ...,"Below is an instruction that describes a task,..."
6998,What does the code raise ?,def call args try return command output args e...,"Below is an instruction that describes a task,..."


In [10]:
generation_config = GenerationConfig(
    temperature=0.1,
    top_p=0.75,
    repetition_penalty=1.1,
    max_new_tokens=256,
)

In [11]:
def generate_response(x):
  input_ids = x.to(model.device)
    
  with torch.no_grad():
      outputs = model.generate(
          input_ids=input_ids,
          generation_config=generation_config,
          return_dict_in_generate=True,
          output_scores=True,
      )
  
  decoded_output = tokenizer.decode(outputs.sequences[0], skip_special_tokens=True)
  response = decoded_output.split("### Response:")[1].strip()
  return response

In [32]:
from tqdm import tqdm
df = pd.read_csv("code_seed_dataset_generated.csv", index_col=0)
df.iloc[0:2000]

with tqdm(total=len(df)) as pbar:
  for idx, row in df.iterrows():
    pbar.update(1)
    if row.isna()["response"]:
      input_ids = tokenizer(row["prompt_input"], return_tensors="pt").input_ids
      response = generate_response(input_ids)
      df.loc[idx, "response"] = response
      if idx % 10 == 0:
        df.to_csv("code_seed_dataset_generated.csv")
        print("Saved to train_df.csv", idx)

,instruction,input,prompt_input,input_ids,response
0,Does it have any filter rules ?,@pytest mark django dbdef test vfolder with no...,"Below is an instruction that describes a task,...","tensor([[ 0, 13866, 338, 385, 15278, ...",No filter rules specified
1,Ca a range generate ranges with a length of mo...,def safe range *args rng xrange *args if len r...,"Below is an instruction that describes a task,...","tensor([[ 0, 13866, 338, 385, 15278, ...",False
2,What ca a range nt generate ?,def safe range *args rng xrange *args if len r...,"Below is an instruction that describes a task,...","tensor([[ 0, 13866, 338, 385, 15278, ...",A range can generate a list of numbers by usin...
3,What ca nt generate ranges with a length of mo...,def safe range *args rng xrange *args if len r...,"Below is an instruction that describes a task,...","tensor([[ 0, 13866, 338, 385, 15278, ...",The function safe range can not generate range...
4,Do method decorator indicate a method is not a...,def not allowed func def inner self *args **kw...,"Below is an instruction that describes a task,...","tensor([[ 0, 13866, 338, 385, 15278, ...","No, it does not indicate that."
...,...,...,...,...,...
1995,What does function add ?,def upgrade migrate engine meta Meta Data bind...,"Below is an instruction that describes a task,...","tensor([[ 0, 13866, 338, 385, 15278, ...",Function add creates a new table and column in...
1996,How should the suffix placed at the end of the...,def suffix handlers default None error ' There...,"Below is an instruction that describes a task,...","tensor([[ 0, 13866, 338, 385, 15278, ...",The suffix passed to the url should be a valid...
1997,What should pass in a dict ?,def suffix handlers default None error ' There...,"Below is an instruction that describes a task,...","tensor([[ 0, 13866, 338, 385, 15278, ...",The following formats are supported by Falcon:...
1998,What does the code get ?,def get best unit unit a unit b a DATETIME UNI...,"Below is an instruction that describes a task,...","tensor([[ 0, 13866, 338, 385, 15278, ...",The code gets the best datetime unit from the ...


In [20]:
pd.read_csv("code_seed_dataset_generated.csv")[["instruction", "input", "response"]]

,instruction,input,response
0,Does it have any filter rules ?,@pytest mark django dbdef test vfolder with no...,No filter rules specified
1,Ca a range generate ranges with a length of mo...,def safe range *args rng xrange *args if len r...,False
2,What ca a range nt generate ?,def safe range *args rng xrange *args if len r...,A range can generate a list of numbers by usin...
3,What ca nt generate ranges with a length of mo...,def safe range *args rng xrange *args if len r...,The function safe range can not generate range...
4,Do method decorator indicate a method is not a...,def not allowed func def inner self *args **kw...,"No, it does not indicate that."
...,...,...,...
6995,What does the code return ?,def get index videos course return list {attr ...,NaN
6996,What contains the substring specified by the a...,def assert has text output text assert output ...,NaN
6997,What does specified output contain ?,def assert has text output text assert output ...,NaN
6998,What does the code raise ?,def call args try return command output args e...,NaN


In [ ]:
# train_df["response"] = train_df["input_ids"].progress_map(generate_response)

In [ ]:
# train_df["refine_response"] = train_df["response"].map(lambda x: x.split("### Response:")[1].strip())
# refine_df = train_df[["instruction", "input", "refine_response"]]

In [ ]:
# refine_df